In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import t, f, norm

In [2]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures

In [3]:
class linear_regression:
    
    def fit(self, X, y):
        X_ = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        Q = np.linalg.inv(X_.T.dot(X_))
        self.beta = Q.dot(X_.T).dot(y)
        
        n = X.shape[0]
        p = X.shape[1]
        self.n_samples = n
        self.n_features = p
        y_hat = X_.dot(self.beta)
        
        self.residual = y - y_hat
        sse = np.square(self.residual).sum()
        ssr = np.square(y_hat - y.mean()).sum()
        sst = np.square(y - y.mean()).sum()
        self.mse = sse/(n-p-1)
        
        self.r_sq = 1 - sse/sst
        self.adj_r = 1 - (sse/(n-p-1))/(sst/(n-1))
        
        self.f_value = (ssr/p) / (sse/(n-p-1))
        self.f_pvalue = 1 - f.cdf(self.f_value, dfn = p, dfd = n - p - 1)
        
        self.std_error = np.sqrt((sse/(n-p-1)) * Q.diagonal())
        self.t_values = self.beta / self.std_error
        self.t_pvalues = [2*(1 - t.cdf(abs(i), n-p-1)) for i in self.t_values]
        
    def predict(self, X):
        X_ = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        return X_.dot(self.beta)
    
    def summary(self, feature_names = None, round_decimals=3, alpha=0.05):
        res_min = self.residual.min()
        res_max = self.residual.max()
        res_1q = np.quantile(self.residual, .25)
        res_med = np.median(self.residual)
        res_3q = np.quantile(self.residual, .75)
        res = pd.DataFrame(np.array([res_min, res_1q, res_med, res_3q, res_max]).reshape(1,-1), columns = ['Residuals: Min', '1Q', 'Median', '3Q', 'Max'])
        
        ci_lower = np.round(self.beta - self.std_error * t.ppf(1-alpha/2, self.n_samples - self.n_features -1), round_decimals)
        ci_upper = np.round(self.beta + self.std_error * t.ppf(1-alpha/2, self.n_samples - self.n_features -1), round_decimals)
        if feature_names is None:
            feat_idx = ['x0(INTCP)'] + ['x{}'.format(i) for i in range(1, n_features+1)]
            coef = pd.DataFrame(np.array([self.beta, self.std_error, self.t_values, np.round(self.t_pvalues, round_decimals), ci_lower, ci_upper]).T, index = feat_idx, columns= 
                               ['coef.', 'std. error', 't_value', 'Pr(>|t|)', '{}% LWR'.format(int(100*(1-alpha))), '{}% UPR'.format(int(100*(1-alpha)))])
        else:
            feat_idx = np.insert(feature_names, 0, '(INTCP)')
            coef = pd.DataFrame(np.array([self.beta, self.std_error, self.t_values, np.round(self.t_pvalues, round_decimals), ci_lower, ci_upper]).T, index = feat_idx, columns= 
                               ['coef.', 'std. error', 't_value', 'Pr(>|t|)', '{}% LWR'.format(int(100*(1-alpha))), '{}% UPR'.format(int(100*(1-alpha)))])
        
        r2 = 'R-squared: {}, Adjusted R-squared: {}'.format(self.r_sq, self.adj_r)
        res_std_err = 'Residual standard error: {} on {} degrees of freedom'.format(self.mse, self.n_samples - self.n_features -1)
        F_stats = 'F-statistic: {} on {} and {} DF, p-value: {}'.format(self.f_value, self.n_features, self.n_samples - self.n_features -1, self.f_pvalue)
        print(res, coef, res_std_err, r2, F_stats, sep='\n'+'='*70+'\n')

# Load data

In [4]:
X, y, feature_names, _, _ = load_boston().values()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X, columns = feature_names), pd.Series(y, name='label'), test_size = .2, random_state=1)

In [6]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train: Linear Regression

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [8]:
lr = linear_regression()
lr.fit(X_train, y_train)
pred_ = lr.predict(X_test)

In [10]:
pred_[1]

28.09349529811247

In [11]:
pred[1]

28.093495298112458

In [12]:
np.allclose(pred,pred_)

True

In [11]:
lr = linear_regression()
lr.fit(X_train, y_train)
lr.summary(feature_names=feature_names)

   Residuals: Min        1Q    Median        3Q        Max
0      -13.392716 -2.713732 -0.537744  1.537035  24.893577
             coef.  std. error    t_value  Pr(>|t|)  95% LWR  95% UPR
(INTCP)  22.522277    0.236767  95.124364     0.000   22.057   22.988
CRIM     -1.026701    0.312280  -3.287755     0.001   -1.641   -0.413
ZN        1.350413    0.375499   3.596319     0.000    0.612    2.089
INDUS     0.125577    0.462665   0.271420     0.786   -0.784    1.035
CHAS      0.575228    0.245493   2.343150     0.020    0.093    1.058
NOX      -2.286092    0.499211  -4.579411     0.000   -3.268   -1.305
RM        2.130839    0.329607   6.464787     0.000    1.483    2.779
AGE       0.127024    0.425805   0.298316     0.766   -0.710    0.964
DIS      -3.178567    0.485752  -6.543602     0.000   -4.134   -2.224
RAD       2.647306    0.675885   3.916797     0.000    1.318    3.976
TAX      -1.877813    0.741576  -2.532191     0.012   -3.336   -0.420
PTRATIO  -2.142964    0.323162  -6.631241 

In [11]:
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     80.85
Date:                Sun, 01 Aug 2021   Prob (F-statistic):          5.53e-102
Time:                        00:36:15   Log-Likelihood:                -1196.4
No. Observations:                 404   AIC:                             2421.
Df Residuals:                     390   BIC:                             2477.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.5223      0.237     95.124      0.0